## This notebook uses plain simple anthropic-sdk based client to do handle the following flow -

1. Notebook is started when an ansible job log completion data comes in.
1. It examimes if there is any error. If no error, it ends
1. If there is an error:
    - Agent analyzes and recommends
    - Agent opens a jira ticket
    - Agent sends a slack message

TODO

- add tools
- provide more description etc

In [1]:
from typing import List
#from util import llm_call
from anthropic import Anthropic
import os
import re
from dotenv import load_dotenv


Agents are the heart of complex AI applications. They combine inference, memory, safety, and tool usage into coherent workflows. At its core, an agent follows a sophisticated execution loop that enables multi-step reasoning, tool usage, and safety checks.
Ref: https://llama-stack.readthedocs.io/en/latest/building_applications/agent_execution_loop.html


In [2]:
# load environment variables from .env
load_dotenv()  

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

def llm_call(prompt: str, system_prompt: str = "", model="claude-3-5-sonnet-20241022") -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".
        model (str, optional): The model to use for the call. Defaults to "claude-3-5-sonnet-20241022".

    Returns:
        str: The response from the language model.
    """
    client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    messages = [{"role": "user", "content": prompt}]
    response = client.messages.create(
        model=model,
        max_tokens=4096,
        system=system_prompt,
        messages=messages,
        temperature=0.1,
    )
    return response.content[0].text


In [3]:
def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = llm_call(f"{prompt}\nInput: {result}")
        print(result)
    return result


def route(input: str) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    #print(f"\nAvailable routes: {list(routes.keys())}")
    is_there_error = f"""
    You can look at the contents of an ansible log and spot if there is any error. 
    If there is any error you will return Error. 
    If there is no error, you will return Success.
    You are only allowed to return 2 words:
    Error
    Success
    
    Input: {input}""".strip()
    
    route_response = llm_call(is_there_error)
    
    print("Observed Status of job:")
    print(route_response)

    return route_response

In [4]:
data_processing_steps = [
    """You can look at the contents of an ansible log and spot the error. 
    You will describe what the error is in a few crisp sentences
    so that a human can take corrective actions.""",
    
    """You will always state -
    I have opened a JIRA ticket, no matter what the input is""",
    
    """You will always state -
    I have slacked the message, no matter what the input is""",
    

]



In [5]:
def flow(input: str,prompts: List[str])-> str:
    route_response = route(input)
    
    if route_response == "Error":
        result = chain(input, prompts)
    else:
        result = "No action Needed"
    return result
    

In [6]:
#could not connect to the host as the password as expired.
#all tasks successfully finished
report = """
could not connect to the host as the password as expired.
"""
print("\nInput text:")
print(report)
result = flow(report,data_processing_steps)


Input text:

could not connect to the host as the password as expired.

Observed Status of job:
Error

Step 1:
Error Analysis:
The SSH connection to the target host failed because the user account's password has expired. This is a common authentication issue that requires the password to be updated before Ansible can successfully connect to and manage the host.

Recommended Action:
Log into the target host directly (or through system administration) and update the expired password. Once the password is reset, update the Ansible credentials accordingly and retry the playbook.

Step 2:
I have opened a JIRA ticket.

Error Analysis:
The SSH connection to the target host failed because the user account's password has expired. This is a common authentication issue that requires the password to be updated before Ansible can successfully connect to and manage the host.

Recommended Action:
Log into the target host directly (or through system administration) and update the expired password. On